In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt

In [2]:
start = dt.datetime(2021,6,1)
end = dt.datetime(2022,1,28)
# stk_data = gh(symbol = 'TATACOFFEE', start=start, end=end)
stk_data = pd.read_csv("Tatacoffee13_21.csv")

In [3]:
stk_data = stk_data[["Open","High","Low","Close"]]
stk_data

,Open,High,Low,Close
0,1410.60,1427.90,1408.30,1415.10
1,1421.00,1626.60,1416.15,1607.40
2,1632.55,1673.90,1613.05,1626.20
3,1627.75,1627.75,1574.60,1579.05
4,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2220,202.90,207.80,201.35,205.00
2221,206.00,206.85,202.05,202.95
2222,203.90,203.90,199.35,201.00
2223,200.00,222.00,196.00,218.35


In [4]:
# column = "Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1=Ms.fit_transform(stk_data)
print("len:",data1.shape)

len: (2225, 4)


In [6]:
data1 = pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [7]:
training_size = round(len(data1)*0.85)
print("training_size-",training_size)
x_train = data1[:training_size]
x_test = data1[training_size:]
print("x_train-",x_train.shape)
print("x_test-",x_test.shape)
y_train = data1[:training_size]
y_test = data1[training_size:]
print("y_train-",y_train.shape)
print("y_test-",y_test.shape)

training_size- 1891
x_train- (1891, 4)
x_test- (334, 4)
y_train- (1891, 4)
y_test- (334, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance = {"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
def combination(dataset,listt):
    print(listt)
    datasetTwo = dataset[listt]
    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order = ',i)
        print('AIC: ',results.aic)
        print('BIC: ',results.bic)
        print()
    x = model.select_order(maxlags=12)
    order = x.selected_orders['aic']
    result = model.fit(order)
    #result.summary()
    lagged_values = train.values[-order:]
    pred = result.forecast(y=lagged_values,steps=28)
    preds = pd.DataFrame(pred,columns = listt)
    preds.to_csv("varforecasted_{}".format(test_obs))
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
    rmse = round(mean_squared_error(test,pred))
    mape = mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf = pd.DataFrame(performance)
    return perf,result,pred

In [11]:
listt = ["Open","High","Low","Close"]

In [12]:
perf,result,pred = combination(data1,listt)

['Open', 'High', 'Low', 'Close']
Order =  1
AIC:  -41.57633250372664
BIC:  -41.52447100279195

Order =  2
AIC:  -41.89965675983833
BIC:  -41.80627099960219

Order =  3
AIC:  -41.89668108122692
BIC:  -41.76173985728735

Order =  4
AIC:  -41.89556182371666
BIC:  -41.71903389353316

Order =  5
AIC:  -41.9246166173129
BIC:  -41.70647070014241

Order =  6
AIC:  -42.00676954931892
BIC:  -41.74697432615162

Order =  7
AIC:  -42.06304859245072
BIC:  -41.76157270594569

Order =  8
AIC:  -42.05891183647394
BIC:  -41.71572389089467

Order =  9
AIC:  -42.075067742123196
BIC:  -41.690136303273015

Order =  10
AIC:  -42.09630316989197
BIC:  -41.66959676504926



In [13]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Open, High, Low, Close]",0,0.088105,11,28


In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
import numpy as np
orders = [(1,2,1),(1,1,2),(2,3,1),(2,2,2)]
from statsmodels.tsa.statespace.sarimax import SARIMAX

for i in orders:
    #print(td)
    model = SARIMAX(data1,order=i,seasonal_order=(0,0,0,12))
    model_fit = model.fit()
    y_pred = model_fit.predict(0,len(data1)-1)
    
    from sklearn.metrics import r2_score
    mse = mean_squared_error(data1, y_pred)
    #print(y_pred)

    print("Order={}".format(i))
    rmse = np.sqrt(mean_squared_error(data1, y_pred))
    print("RMSE:", rmse)

    mape = mean_absolute_percentage_error(data1, y_pred)
    print("MAPE:", mape)
    print("***********")

In [ ]:
len(y_pred)

In [ ]:
order=(1, 1, 2)
model = SARIMAX(data1,order=order,seasonal_order=(0,0,0,12))
model_fit = model.fit()
y_pred = model_fit.predict(0,len(data1)-1)
from sklearn.metrics import r2_score
mse = mean_squared_error(data1,y_pred)
print("Order={}".format(order))
rmse = np.sqrt(mean_squared_error(data1, y_pred))
print("RMSE:", rmse)
mape = mean_absolute_percentage_error(data1, y_pred)
print("MAPE:", mape)

In [ ]:
from stockFunctions import graph
graph(data1,y_pred,"Actual","Predicted","TataCoffee-Close-AR-Norm","Days","Prices")

In [ ]:
from stockFunctions import conversionSingle
aTestNormTable=conversionSingle(data1,[column])
actual_stock_price_test_ori = Ms.inverse_transform(aTestNormTable)
actual_stock_price_test_oriA = conversionSingle(actual_stock_price_test_ori,[column])

In [ ]:
pTestNormTable=conversionSingle(y_pred,[column])
predicted_stock_price_test_ori = Ms.inverse_transform(pTestNormTable)
predicted_stock_price_test_oriA = conversionSingle(predicted_stock_price_test_ori,[column])

In [ ]:
graph(actual_stock_price_test_oriA,predicted_stock_price_test_oriA,"Actual","Predicted","TataCoffee-Close-AR-Norm","Days","Prices")

In [ ]:
rmse = np.sqrt(mean_squared_error(actual_stock_price_test_oriA, predicted_stock_price_test_oriA))
print("RMSE:", rmse)
mape = mean_absolute_percentage_error(actual_stock_price_test_oriA, predicted_stock_price_test_oriA)
print("MAPE:", mape)

In [ ]:
forecast = model_fit.predict(len(data1),len(data1)+3)
forecast

In [ ]:
fTestNormTable=conversionSingle(forecast,["Closefore"])
forecast_stock_price_test_ori = Ms.inverse_transform(fTestNormTable)
forecast_stock_price_test_oriA = conversionSingle(forecast_stock_price_test_ori,["Closefore"])
forecast_stock_price_test_oriA

In [ ]:
forecast_stock_price_test_oriA.to_csv("CloseSARIMA.csv",index=False)